## **네이버뉴스 카카오톡 전송 프로젝트**

### **모듈 패키지 설치**

In [ ]:
!pip install gensim BeautifulSoup4 lxml requests selenium

     |████████████████████████████████| 963 kB 8.4 MB/s 
     |████████████████████████████████| 138 kB 47.7 MB/s 
     |████████████████████████████████| 359 kB 74.1 MB/s 
     |████████████████████████████████| 138 kB 53.7 MB/s 
     |████████████████████████████████| 138 kB 80.4 MB/s 
     |████████████████████████████████| 138 kB 80.8 MB/s 
     |████████████████████████████████| 153 kB 78.5 MB/s 
     |████████████████████████████████| 137 kB 76.2 MB/s 
     |████████████████████████████████| 136 kB 82.6 MB/s 
     |████████████████████████████████| 136 kB 70.6 MB/s 
     |████████████████████████████████| 136 kB 69.5 MB/s 
INFO: pip is looking at multiple versions of trio-websocket to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of attrs to determine which version is compatible with other requirements. This could take a while.
     |████████████████████████████████| 60 kB 9.6 MB/s 
     |████████

### **네이버뉴스 url 크롤링하기**

In [ ]:
import requests
from bs4 import BeautifulSoup
import bs4.element
import datetime
import json

# BeautifulSoup 객체 생성
def get_soup_obj(url):
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
    res = requests.get(url, headers = headers)
    soup = BeautifulSoup(res.text,'lxml')
    
    return soup

# 뉴스의 기본 정보 가져오기
def get_top3_news_info(sec, sid):
    # 임시 이미지
    default_img = "https://search.naver.com/search.naver?where=image&sm=tab_jum&query=naver#"
    
     # 해당 분야 상위 뉴스 목록 주소
    sec_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec" \
                + "&sid1=" \
                + sid
    print("section url : ", sec_url)
    
    # 해당 분야 상위 뉴스 HTML 가져오기
    soup = get_soup_obj(sec_url)
  
    # 해당 분야 상위 뉴스 3개 가져오기
    news_list3 = []
    lis3 = soup.find('ul', class_='type06_headline').find_all("li", limit=3)
    
    for li in lis3:
        # title : 뉴스 제목, news_url : 뉴스 URL, image_url : 이미지 URL
        news_info = {
            "title" : li.img.attrs.get('alt') if li.img else li.a.text.replace("\n", "").replace("\t","").replace("\r","") , 
            "date" : li.find(class_="date").text,
            "news_url" : li.a.attrs.get('href'),
            "image_url" :  li.img.attrs.get('src') if li.img else default_img
        }
        news_list3.append(news_info)
        
    return news_list3

In [ ]:
# 경제', '사회', 'IT' 분야의 상위 3개 뉴스 크롤링
def get_naver_news_top3():
    # 뉴스 결과를 담아낼 dictionary
    news_dic = dict()
    
    # sections : 경제', '사회', "IT"
    sections = ["eco", "soc", "IT"]
    # section_ids :  URL에 사용될 뉴스  각 부문 ID
    section_ids = ["101", "102", "105"]
    
    for sec, sid in zip(sections, section_ids):   
        # 뉴스의 기본 정보 가져오기
        news_info = get_top3_news_info(sec, sid)
        # print(news_info)
        for news in news_info:
            # 뉴스 url 가져오기
            news_url = news['news_url']

        news_dic[sec] = news_info
    
    return news_dic
    
# 함수 호출 - '경제', '사회', 'IT' 분야의 상위 3개 뉴스 크롤링
# news_dic 에 분야별로 3개씩 기사 넣기
news_dic = get_naver_news_top3()

print("*"*100)

# news_dic 결과 확인하기 
news_dic

section url :  https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=101


AttributeError: ignored

### **gensim뉴스 요약 #1**

In [ ]:
# 1
from gensim.summarization.summarizer import summarize

# 섹션 지정 (경제, 사회, IT중 하나를 입력)
my_section = input('eco, soc, IT 중 하나를 입력하세요 : ')
news_list3 = news_dic[my_section]

# 뉴스 요약하기
for news_info in news_list3:
    
    # 뉴스 타이틀이 20 문자 초과일 경우 생략. 즉, 최대 20문자까지만 출력.
    try:
        snews_contents = summarize(news_info['title'], word_count=20)
    except:
        snews_contents = None

    news_info['snews_contents'] = snews_contents

# 요약한 news_list3 출력    
news_list3

eco, soc, IT 중 하나를 입력하세요 : IT


NameError: ignored

### **카카오톡 토큰 발급**

In [ ]:
# 카카오 개발자 사이트
print("https://developers.kakao.com/docs/latest/ko/kakaologin/rest-api#request-code")

# rest api키
# e677cd10264d501d71000b7fc4a68d40

# 인증코드 받기
# https://kauth.kakao.com/oauth/authorize?client_id=e677cd10264d501d71000b7fc4a68d40&response_type=code&redirect_uri=https://localhost.com

https://developers.kakao.com/docs/latest/ko/kakaologin/rest-api#request-code


In [ ]:
import requests
import json

url = "https://kauth.kakao.com/oauth/token"

data = {
    "grant_type" : "authorization_code",
    "client_id" : "e677cd10264d501d71000b7fc4a68d40",
    "redirect_uri" : "https://localhost.com",
    "code"         : "_Oyc9PDzfiINrq0gMLGj8Fc5IFKv_deaUvFPP1eBDv4crFvhTNVcZDZHKHh0rDjNfuZqqwo9dGgAAAF7ynjwdw"
    
}
response = requests.post(url, data=data)

tokens = response.json()

print(tokens)

{'access_token': 'EjDWb0hTuE4vJFRQLGCUWYEEWqprpmZ1tZZHxwo9dVwAAAF7ynmAeg', 'token_type': 'bearer', 'refresh_token': 'x_OZcozdbQ15o40EthOYb3gsYkm1-HXdfxcSXwo9dVwAAAF7ynmAeQ', 'expires_in': 21599, 'scope': 'account_email talk_message profile_nickname', 'refresh_token_expires_in': 5183999}


In [ ]:
with open("kakao_token.json", "w") as fp:
    json.dump(tokens, fp)

In [ ]:
# 모듈 패키지 불러와서 저장하기

from google.colab import files
src = list(files.upload().values())[0]
open('file1.py','wb').write(src)
import file1

Saving kakao_utils.py to kakao_utils.py


In [ ]:
# 생성한 토큰 접근하기

import json
import kakao_utils

KAKAO_TOKEN_FILENAME = "/content/kakao_token.json" 
KAKAO_APP_KEY = "e677cd10264d501d71000b7fc4a68d40"
kakao_utils.update_tokens(KAKAO_APP_KEY, KAKAO_TOKEN_FILENAME)

{'access_token': 'fnk8vxjR6RyGU7Bedx1fvIHfrmbW9SFjHSOcnQorDR8AAAF7ynnFdA', 'token_type': 'bearer', 'expires_in': 21599}


{'access_token': 'fnk8vxjR6RyGU7Bedx1fvIHfrmbW9SFjHSOcnQorDR8AAAF7ynnFdA',
 'expires_in': 21599,
 'refresh_token': 'x_OZcozdbQ15o40EthOYb3gsYkm1-HXdfxcSXwo9dVwAAAF7ynmAeQ',
 'refresh_token_expires_in': 5183999,
 'scope': 'account_email talk_message profile_nickname',
 'token_type': 'bearer'}

### **템플릿 만들기 #2**

In [ ]:
# 2 
# 사용자가 선택한 카테고리를 제목에 넣기 위한 dictionary
sections_ko = {'eco' : '경제','soc' : '사회', 'IT' : 'IT/과학'}

# 네이버 뉴스 URL
navernews_url = "https://news.naver.com/main/home.nhn"

# 추후 각 리스트에 들어갈 내용(content) 만들기
contents = []

# 리스트 템플릿 형식 만들기
template = {
    "object_type" : "list",
    "header_title" : sections_ko[my_section] + " 분야 상위 뉴스 빅3",
    "header_link" : {
        "web_url": navernews_url,
        "mobile_web_url" : navernews_url
    },
    "contents" : contents,
    "button_title" : "네이버 뉴스 바로가기"
}

## 내용 만들기
# 각 리스트에 들어갈 내용(content) 만들기
for news_info in news_list3:    
    content = {
        "title" : news_info.get('title'),
        "description" : "작성일 : " + news_info.get('date'),
        "image_url" : news_info.get('image_url'),
        "image_width" : 50, "image_height" : 50,
        "link": {
            "web_url": news_info.get('news_url'),
            "mobile_web_url": news_info.get('news_url')
        }
    }
    
    contents.append(content)
    

### **카카오 메시지 전송 코드 #3**

In [ ]:
# 3
# 카카오 메시지 전송
res = kakao_utils.send_message(KAKAO_TOKEN_FILENAME, template)
if res.json().get('result_code') == 0:
    print('뉴스를 성공적으로 보냈습니다.')
else:
    print('뉴스를 성공적으로 보내지 못했습니다. 오류메시지 : ', res.json())

분야별 뉴스기사 보낼때 : 1.gensim뉴스요약 -> 2.템플릿 만들기 -> 3.카카오 메시지

## **번외**

### **뉴스 속보 3개 추출**

In [ ]:
# 3번에 걸쳐 각 뉴스 속보 요약 결과를 전송합니다
for idx, news_info in enumerate(news_list3):
    # 텍스트 템플릿 형식 만들기
    template = {
      "object_type": "text",
      "text": '# 제목 : ' + news_info.get('title'),
      "link": {
            "web_url": news_info.get('news_url'),
            "mobile_web_url": news_info.get('news_url')
      },
      "button_title": "자세히 보기"
    }
    
    # 카카오 메시지 전송
    res = kakao_utils.send_message(KAKAO_TOKEN_FILENAME, template)
    if res.json().get('result_code') == 0:
        print('뉴스를 성공적으로 보냈습니다.')
    else:
        print('뉴스를 성공적으로 보내지 못했습니다. 오류메시지 : ', res.json())

뉴스를 성공적으로 보냈습니다.
뉴스를 성공적으로 보냈습니다.
뉴스를 성공적으로 보냈습니다.


### **코로나 발생현황 메시지 추출**

In [ ]:
from bs4 import BeautifulSoup
import requests

covid_url = "http://ncov.mohw.go.kr/"
req = requests.get(covid_url) # 코로나바이러스 사이트
# print(req.text)

soup = BeautifulSoup(req.text,"html.parser")

domestic = soup.find("div", class_ = "liveNum_today_new").find_all("span",class_="data")[0] # 국내발생
abroad = soup.find("div", class_ = "liveNum_today_new").find_all("span",class_="data")[1] # 해외유입

domestic_num = domestic.text.replace(",","")
abroad_num = abroad.text.replace(",","")


covid_sum = int(domestic_num) + int(abroad_num)

covid_li = "국내 발생 : " + domestic.text + " | " + "해외 발생 : " + abroad.text + " | " + "합계 : " + str(covid_sum)

print(covid_li)

국내 발생 : 2,014 | 해외 발생 : 36 | 합계 : 2050


In [ ]:
template = {
    "object_type" : "text",
    "title" : "코로나 발생 현황",
    "text" : covid_li,
    "link" : {
        "web_url": "http://ncov.mohw.go.kr/",
        "mobile_web_url" : "http://ncov.mohw.go.kr/"
    }
}

In [ ]:
res = kakao_utils.send_message(KAKAO_TOKEN_FILENAME, template)
if res.json().get('result_code') == 0:
    print('코로나를 성공적으로 보냈습니다.')
else:
    print('코로나를 성공적으로 보내지 못했습니다. 오류메시지 : ', res.json())

코로나를 성공적으로 보냈습니다.
